In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras

from sklearn.model_selection import RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
trainDataset = pd.read_csv("/content/fashion_mnist_train.csv")
testDataset = pd.read_csv("/content/fashion_mnist_test.csv")

XTrain = trainDataset.iloc[:, 1:785]
YTrain = trainDataset.iloc[:, 0:1]

XTest = testDataset.iloc[:, 1:785]
YTest = testDataset.iloc[:, 0:1]

YCategorizedTrain = keras.utils.to_categorical(YTrain);
YCategorizedTest = keras.utils.to_categorical(YTest);

In [ ]:
def createModel (layers=5, neurons=16, alpha=1.0e-05, activation='relu'):
    model = Sequential()
    model.add(Input(shape=(784,)))
    for i in range(layers):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(10, activation='softmax'))
    optimizer = Adam(learning_rate=alpha)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

    return model

model = KerasClassifier(build_fn=createModel)

In [ ]:
layers = list(range(5,31))
activation = ['relu', 'tanh', 'softmax', 'softplus', 'sigmoid']
neurons = list(range(16,1024))
alpha = [0.01, 0.001, 0.0001, 0.00001]
parameters = dict(layers=layers, activation=activation, neurons=neurons, alpha=alpha)

randomized = RandomizedSearchCV(estimator=model, param_distributions=parameters, cv=3, verbose=2, n_iter=10)
randomized.fit(XTrain, YCategorizedTrain)
print("Best Parameters: ", randomized.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] neurons=662, layers=7, alpha=1e-05, activation=tanh .............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


625/625 [==============================] - 6s 9ms/step - loss: 0.4980 - accuracy: 0.8227
[CV]  neurons=662, layers=7, alpha=1e-05, activation=tanh, total=  49.2s
[CV] neurons=662, layers=7, alpha=1e-05, activation=tanh .............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   49.2s remaining:    0.0s


625/625 [==============================] - 6s 9ms/step - loss: 0.4962 - accuracy: 0.8171
[CV]  neurons=662, layers=7, alpha=1e-05, activation=tanh, total=  49.7s
[CV] neurons=662, layers=7, alpha=1e-05, activation=tanh .............
625/625 [==============================] - 6s 9ms/step - loss: 0.5033 - accuracy: 0.8178
[CV]  neurons=662, layers=7, alpha=1e-05, activation=tanh, total=  48.3s
[CV] neurons=261, layers=24, alpha=1e-05, activation=relu ............
625/625 [==============================] - 3s 5ms/step - loss: 0.6011 - accuracy: 0.7869
[CV]  neurons=261, layers=24, alpha=1e-05, activation=relu, total=  35.0s
[CV] neurons=261, layers=24, alpha=1e-05, activation=relu ............
625/625 [==============================] - 3s 5ms/step - loss: 0.6343 - accuracy: 0.7669
[CV]  neurons=261, layers=24, alpha=1e-05, activation=relu, total=  34.5s
[CV] neurons=261, layers=24, alpha=1e-05, activation=relu ............
625/625 [==============================] - 3s 5ms/step - loss: 0.5

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 18.6min finished


1875/1875 [==============================] - 42s 22ms/step - loss: 2.2467 - accuracy: 0.7407
Best Parameters:  {'neurons': 572, 'layers': 6, 'alpha': 0.001, 'activation': 'relu'}


In [ ]:
bestModel = randomized.best_estimator_
YPred = bestModel.predict(XTest)

accuracy = 0
YPred = list(YPred)
YTest = list(YTest['label'])
for i in range(len(YPred)):
    if YPred[i] == YTest[i]:
        accuracy = accuracy + 1
accuracy = (accuracy/len(YPred))*100
print("Accuracy: ", accuracy) 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy:  84.11
